In [178]:
import numpy as np
import pandas as pd
import os
import shutil
import sklearn.decomposition
# import config_utils
# import argparse

In [179]:
outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/interaction_qtl/timepoint_D52_D30/input_files/celltype_DA/"
# outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/interaction_qtl/timepoint_D52_D30/input_files/celltype_Sert/"
# outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/interaction_qtl/timepoint_D52_D30/input_files/celltype_Epen1/"
# outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/interaction_qtl/timepoint_D52_D30/input_files/celltype_Astro/"

In [180]:
# outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/interaction_qtl/timepoint_D30_D52/input_files/celltype_DA/"
# outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/interaction_qtl/timepoint_D30_D52/input_files/celltype_Sert/"
# outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/interaction_qtl/timepoint_D30_D52/input_files/celltype_Epen1/"
# outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/interaction_qtl/timepoint_D30_D52/input_files/celltype_Astro/"

In [181]:
# outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/interaction_qtl/timepoint_D30_D52/input_files_ROT/celltype_DA/"
# outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/interaction_qtl/timepoint_D30_D52/input_files_ROT/celltype_Sert/"
# outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/interaction_qtl/timepoint_D30_D52/input_files_ROT/celltype_Epen1/"
# outdir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/interaction_qtl/timepoint_D30_D52/input_files_ROT/celltype_Astro/"

In [182]:
# general set up
genotypes_file = '/hps/nobackup/hipsci/scratch/genotypes/imputed/REL-2018-01/Full_Filtered_Plink-f/hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.recode.filtered'
kinship_file = '/hps/nobackup/hipsci/scratch/genotypes/imputed/REL-2018-01/Full_Filtered_Plink-f/hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.recode.filtered.rel'
annotation_file = '/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_processed/scQTLs/annos/ensembl_gene_id_annos.tsv'
#annotation_file = '/hps/nobackup/hipsci/scratch/singlecell_endodiff/data_processed/scQTLs/annos/hgnc_symbol_annos.tsv'
chunk_file = '/nfs/leia/research/stegle/mjbonder/ChunkFiles/Ensembl_75_Limix_Annotation_FC_Gene_step10.txt'
n_pcs = 10
base_dir = '/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/interaction_qtl/timepoint_D52_D30/'

In [183]:
# fvf_file = '/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/interaction_qtl/feature_variant_filter.tsv'
fvf_file = '/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/eqtl/feature_variant_filter.tsv'

In [184]:
sample_mapping_file = os.path.join(outdir, 'samplemapping.tsv')
phenotype_file = os.path.join(outdir, 'phenotypes.tsv')
covariates_file = os.path.join(outdir, 'covariates.tsv')
# noise_matrix_file = os.path.join(outdir, 'noise_matrix.tsv')
# config_file = os.path.join(outdir, 'qtl_config.yaml')

In [185]:
if not os.path.exists(outdir):
    os.makedirs(outdir, exist_ok=True)

In [186]:
#load map HGNC to ensembl
mapping_df = pd.read_csv('/nfs/leia/research/stegle/dseaton/genomes/hg19/annotation/geneid_mappings/hgnc_symbol2ensembl_gene_id.txt', sep='\t')

In [187]:
mydir = '/nfs/leia/research/stegle/dseaton/hipsci/singlecell_neuroseq/data/data_processed/'

In [188]:
# # ## DA
D30 = 'pool1_13_noddd_D30/pool1_13_noddd_D30.scanpy.w_metadata.w_celltype.scanpy.celltype.DA.groupedby.donor_id-celltype.mean.tsv'
D52 = 'pool1_17_D52/pool1_17_D52.scanpy.w_metadata.w_celltype.scanpy.celltype.DA.groupedby.donor_id-celltype-treatment.mean.tsv'
# D52 = 'pool1_17_D52/pool1_17_D52.scanpy.w_metadata.w_celltype.scanpy.celltype.DA.groupedby.donor_id-celltype.mean.tsv'

In [189]:
# ## Sert
# D30 = 'pool1_13_noddd_D30/pool1_13_noddd_D30.scanpy.w_metadata.w_celltype.scanpy.celltype.Sert.groupedby.donor_id-celltype.mean.tsv'
# D52 = 'pool1_17_D52/pool1_17_D52.scanpy.w_metadata.w_celltype.scanpy.celltype.Sert.groupedby.donor_id-celltype-treatment.mean.tsv'
# D52 = 'pool1_17_D52/pool1_17_D52.scanpy.w_metadata.w_celltype.scanpy.celltype.Sert.groupedby.donor_id-celltype.mean.tsv'

In [190]:
# ## Epen1
# D30 = 'pool1_13_noddd_D30/pool1_13_noddd_D30.scanpy.w_metadata.w_celltype.scanpy.celltype.Epen1.groupedby.donor_id-celltype.mean.tsv'
# D52 = 'pool1_17_D52/pool1_17_D52.scanpy.w_metadata.w_celltype.scanpy.celltype.Epen1.groupedby.donor_id-celltype-treatment.mean.tsv'
# D52 = 'pool1_17_D52/pool1_17_D52.scanpy.w_metadata.w_celltype.scanpy.celltype.Epen1.groupedby.donor_id-celltype.mean.tsv'

In [191]:
## Astro
# D30 = 'pool1_13_noddd_D30/pool1_13_noddd_D30.scanpy.w_metadata.w_celltype.scanpy.celltype.Astro.groupedby.donor_id-celltype.mean.tsv'
# D52 = 'pool1_17_D52/pool1_17_D52.scanpy.w_metadata.w_celltype.scanpy.celltype.Astro.groupedby.donor_id-celltype-treatment.mean.tsv'
# D52 = 'pool1_17_D52/pool1_17_D52.scanpy.w_metadata.w_celltype.scanpy.celltype.Astro.groupedby.donor_id-celltype.mean.tsv'

In [192]:
phenotype_input_file1 = mydir + D30
phenotype_input_file2 = mydir + D52

In [193]:
phenotype_input_file1

'/nfs/leia/research/stegle/dseaton/hipsci/singlecell_neuroseq/data/data_processed/pool1_13_noddd_D30/pool1_13_noddd_D30.scanpy.w_metadata.w_celltype.scanpy.celltype.DA.groupedby.donor_id-celltype.mean.tsv'

In [194]:
# process phenotype dataframe
groupedby_df1 = pd.read_csv(phenotype_input_file1, sep='\t')
groupedby_df1.head()

,donor_id,celltype,n_cells,AP006222.2,RP11-206L10.9,LINC00115,FAM41C,RP11-54O7.1,RP11-54O7.3,SAMD11,...,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC011043.1,AL592183.1,SRSF10-1
0,HPSI0114i-bezi_3,DA,115,0.198326,0.047601,0.000000,0.018117,0.022951,0.013092,0.508570,...,3.008669,1.594028,0.565621,2.864180,1.382154,0.089891,2.879348,0.101191,0.368187,0.152762
1,HPSI0114i-eipl_1,DA,51,0.224686,0.000000,0.012630,0.043066,0.027141,0.010593,0.368389,...,3.053793,1.783044,0.456721,3.013608,1.477265,0.129621,2.946993,0.000000,0.000000,0.123257
2,HPSI0114i-kolf_2,DA,249,0.142147,0.006062,0.016913,0.017095,0.012296,0.012415,0.359551,...,2.555073,0.603660,0.261821,2.984659,1.092945,0.176214,1.934296,0.039779,0.029085,0.024163
3,HPSI0114i-lexy_2,DA,54,0.192752,0.016041,0.000000,0.000000,0.000000,0.015229,0.548295,...,2.818294,1.828924,0.531615,2.726100,1.314751,0.083156,2.975139,0.017032,0.023986,0.012638
4,HPSI0114i-posc_1,DA,79,0.279501,0.000000,0.011469,0.027462,0.020733,0.010202,0.276795,...,2.912263,1.526831,0.364194,2.689908,1.393556,0.061592,2.805627,0.033783,0.391996,0.080379


In [195]:
# process phenotype dataframe
groupedby_df2 = pd.read_csv(phenotype_input_file2, sep='\t')
groupedby_df2.head()

,donor_id,celltype,treatment,n_cells,AP006222.2,RP11-206L10.3,RP11-206L10.9,LINC00115,FAM41C,RP11-54O7.1,...,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC011043.1,AL592183.1,SRSF10-1
0,HPSI0114i-bezi_3,DA,NONE,66,0.373148,0.010521,0.034687,0.012997,0.014373,0.024058,...,3.452707,1.843407,0.599258,3.306375,2.241297,0.292286,3.334886,0.049472,0.382724,0.076166
1,HPSI0114i-bezi_3,DA,ROT,37,0.448632,0.000000,0.000000,0.029074,0.019211,0.000000,...,2.400357,0.487581,0.452487,2.850002,1.746798,0.075562,2.372990,0.100227,0.430272,0.046033
2,HPSI0114i-eipl_1,DA,NONE,189,0.362936,0.006138,0.015826,0.026636,0.023932,0.013544,...,3.618761,2.037559,0.564256,3.442609,2.081173,0.198851,3.564958,0.000000,0.000000,0.093761
3,HPSI0114i-eipl_1,DA,ROT,59,0.337387,0.000000,0.017640,0.061216,0.007515,0.011137,...,2.724298,0.872619,0.670583,3.302946,1.836064,0.101823,2.852144,0.000000,0.000000,0.016806
4,HPSI0114i-fikt_3,DA,NONE,26,0.269007,0.000000,0.062557,0.000000,0.000000,0.000000,...,3.209428,2.040483,0.669133,3.445064,2.108070,0.153443,3.353447,0.032703,0.161917,0.102277


In [196]:
#subset columns
groupedby_df1 = groupedby_df1.iloc[:, :]
if 'time_point' in groupedby_df1.columns:
    groupedby_df1 = groupedby_df.query('time_point=="D30"')
groupedby_df1.head()

,donor_id,celltype,n_cells,AP006222.2,RP11-206L10.9,LINC00115,FAM41C,RP11-54O7.1,RP11-54O7.3,SAMD11,...,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC011043.1,AL592183.1,SRSF10-1
0,HPSI0114i-bezi_3,DA,115,0.198326,0.047601,0.000000,0.018117,0.022951,0.013092,0.508570,...,3.008669,1.594028,0.565621,2.864180,1.382154,0.089891,2.879348,0.101191,0.368187,0.152762
1,HPSI0114i-eipl_1,DA,51,0.224686,0.000000,0.012630,0.043066,0.027141,0.010593,0.368389,...,3.053793,1.783044,0.456721,3.013608,1.477265,0.129621,2.946993,0.000000,0.000000,0.123257
2,HPSI0114i-kolf_2,DA,249,0.142147,0.006062,0.016913,0.017095,0.012296,0.012415,0.359551,...,2.555073,0.603660,0.261821,2.984659,1.092945,0.176214,1.934296,0.039779,0.029085,0.024163
3,HPSI0114i-lexy_2,DA,54,0.192752,0.016041,0.000000,0.000000,0.000000,0.015229,0.548295,...,2.818294,1.828924,0.531615,2.726100,1.314751,0.083156,2.975139,0.017032,0.023986,0.012638
4,HPSI0114i-posc_1,DA,79,0.279501,0.000000,0.011469,0.027462,0.020733,0.010202,0.276795,...,2.912263,1.526831,0.364194,2.689908,1.393556,0.061592,2.805627,0.033783,0.391996,0.080379


In [197]:
#subset columns (only keep non treated D52 cells in this instance)
groupedby_df2 = groupedby_df2.iloc[:, :]
if 'time_point' in groupedby_df2.columns:
    groupedby_df2 = groupedby_df.query('time_point=="D52"')
groupedby_df2 = groupedby_df2.query('treatment=="NONE"') # or not
groupedby_df2.head()

,donor_id,celltype,treatment,n_cells,AP006222.2,RP11-206L10.3,RP11-206L10.9,LINC00115,FAM41C,RP11-54O7.1,...,MT-CO3,MT-ND3,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC011043.1,AL592183.1,SRSF10-1
0,HPSI0114i-bezi_3,DA,NONE,66,0.373148,0.010521,0.034687,0.012997,0.014373,0.024058,...,3.452707,1.843407,0.599258,3.306375,2.241297,0.292286,3.334886,0.049472,0.382724,0.076166
2,HPSI0114i-eipl_1,DA,NONE,189,0.362936,0.006138,0.015826,0.026636,0.023932,0.013544,...,3.618761,2.037559,0.564256,3.442609,2.081173,0.198851,3.564958,0.000000,0.000000,0.093761
4,HPSI0114i-fikt_3,DA,NONE,26,0.269007,0.000000,0.062557,0.000000,0.000000,0.000000,...,3.209428,2.040483,0.669133,3.445064,2.108070,0.153443,3.353447,0.032703,0.161917,0.102277
6,HPSI0114i-kolf_2,DA,NONE,547,0.334929,0.008299,0.019432,0.017822,0.021774,0.019566,...,3.694610,1.686145,0.529644,3.518382,2.384729,0.149548,3.190676,0.055149,0.033962,0.005144
8,HPSI0114i-lexy_2,DA,NONE,96,0.249292,0.009494,0.017764,0.017159,0.000000,0.000000,...,3.340785,2.379478,0.704406,3.266583,1.847233,0.291636,3.339195,0.000000,0.028323,0.077142


In [198]:
## add time point information to separate 
groupedby_df1['timepoint'] = "D30"
groupedby_df1['treatment'] = "NONE"
groupedby_df2['timepoint'] = "D52"

In [199]:
groupedby_df = pd.concat([groupedby_df1,groupedby_df2], join='inner')
groupedby_df

,donor_id,celltype,n_cells,AP006222.2,RP11-206L10.9,LINC00115,FAM41C,RP11-54O7.1,RP11-54O7.3,SAMD11,...,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC011043.1,AL592183.1,SRSF10-1,timepoint,treatment
0,HPSI0114i-bezi_3,DA,115,0.198326,0.047601,0.000000,0.018117,0.022951,0.013092,0.508570,...,0.565621,2.864180,1.382154,0.089891,2.879348,0.101191,0.368187,0.152762,D30,NONE
1,HPSI0114i-eipl_1,DA,51,0.224686,0.000000,0.012630,0.043066,0.027141,0.010593,0.368389,...,0.456721,3.013608,1.477265,0.129621,2.946993,0.000000,0.000000,0.123257,D30,NONE
2,HPSI0114i-kolf_2,DA,249,0.142147,0.006062,0.016913,0.017095,0.012296,0.012415,0.359551,...,0.261821,2.984659,1.092945,0.176214,1.934296,0.039779,0.029085,0.024163,D30,NONE
3,HPSI0114i-lexy_2,DA,54,0.192752,0.016041,0.000000,0.000000,0.000000,0.015229,0.548295,...,0.531615,2.726100,1.314751,0.083156,2.975139,0.017032,0.023986,0.012638,D30,NONE
4,HPSI0114i-posc_1,DA,79,0.279501,0.000000,0.011469,0.027462,0.020733,0.010202,0.276795,...,0.364194,2.689908,1.393556,0.061592,2.805627,0.033783,0.391996,0.080379,D30,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,HPSI1113i-vuud_2,DA,406,0.375452,0.013560,0.011352,0.012704,0.044862,0.018262,0.436954,...,1.377664,3.506058,2.709661,0.273023,3.600350,0.055638,0.170650,0.007517,D52,NONE
326,HPSI1114i-kuul_1,DA,572,0.352503,0.009886,0.009035,0.006936,0.023690,0.022639,0.392022,...,0.586040,3.423439,2.261969,0.256890,3.650171,0.040104,0.076292,0.004784,D52,NONE
328,HPSI1114i-ualf_6,DA,14,0.364334,0.000000,0.000000,0.056234,0.041154,0.000000,0.605683,...,1.316309,3.362643,2.720644,0.363266,3.327422,0.000000,0.392237,0.117778,D52,NONE
329,HPSI1114i-zuuy_4,DA,25,0.605280,0.025472,0.122520,0.063463,0.000000,0.014755,0.111632,...,0.257385,3.227717,1.970693,0.034769,3.499956,0.043750,0.000000,0.000000,D52,NONE


In [200]:
#apply filter to number of cells per sample (100)
n_cell_threshold = 100
groupedby_df = groupedby_df.query('n_cells>=@n_cell_threshold')

In [201]:
#create merged index
groupedby_df['index'] = groupedby_df['donor_id'] + '_' + groupedby_df['timepoint']
cell_count_ds = groupedby_df['n_cells']
cell_count_ds.index = groupedby_df['index'].tolist()

/nfs/software/stegle/users/acuomo/conda-envs/mypy3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [202]:
cell_count_ds

HPSI0114i-bezi_3_D30    115
HPSI0114i-kolf_2_D30    249
HPSI0114i-wegi_1_D30    603
HPSI0115i-aion_2_D30    448
HPSI0115i-aoxv_1_D30    257
                       ... 
HPSI1113i-qolg_1_D52    131
HPSI1113i-qorq_1_D52    274
HPSI1113i-vuud_2_D52    406
HPSI1114i-kuul_1_D52    572
HPSI1213i-hehd_1_D52    480
Name: n_cells, Length: 214, dtype: int64

In [203]:
#take donor and pool cols for samplemapping df
samplemapping_df = groupedby_df[['donor_id','index']]
samplemapping_df.to_csv(sample_mapping_file, sep='\t', index=False, header=False)

In [204]:
#reorganise to just be expression data indexed by the merged index
groupedby_df = groupedby_df.set_index('index')
groupedby_df.head()

,donor_id,celltype,n_cells,AP006222.2,RP11-206L10.9,LINC00115,FAM41C,RP11-54O7.1,RP11-54O7.3,SAMD11,...,MT-ND4L,MT-ND4,MT-ND5,MT-ND6,MT-CYB,AC011043.1,AL592183.1,SRSF10-1,timepoint,treatment
index,,,,,,,,,,,,,,,,,,,,,
HPSI0114i-bezi_3_D30,HPSI0114i-bezi_3,DA,115,0.198326,0.047601,0.000000,0.018117,0.022951,0.013092,0.508570,...,0.565621,2.864180,1.382154,0.089891,2.879348,0.101191,0.368187,0.152762,D30,NONE
HPSI0114i-kolf_2_D30,HPSI0114i-kolf_2,DA,249,0.142147,0.006062,0.016913,0.017095,0.012296,0.012415,0.359551,...,0.261821,2.984659,1.092945,0.176214,1.934296,0.039779,0.029085,0.024163,D30,NONE
HPSI0114i-wegi_1_D30,HPSI0114i-wegi_1,DA,603,0.160197,0.006713,0.008016,0.012269,0.007431,0.012523,0.439865,...,0.186442,2.617375,1.112830,0.139301,2.883351,0.053963,0.024837,0.081625,D30,NONE
HPSI0115i-aion_2_D30,HPSI0115i-aion_2,DA,448,0.213156,0.012346,0.006512,0.023763,0.026310,0.023601,0.400499,...,0.290392,2.559630,1.327861,0.082398,2.910592,0.119339,0.004075,0.170708,D30,NONE
HPSI0115i-aoxv_1_D30,HPSI0115i-aoxv_1,DA,257,0.193307,0.005231,0.003672,0.018983,0.027040,0.023279,0.362280,...,0.246172,2.961238,1.269954,0.120681,3.154951,0.057383,0.135736,0.087724,D30,NONE


In [205]:
phenotype_df = groupedby_df.drop(['donor_id','celltype','treatment','n_cells','timepoint'], axis=1)
# phenotype_df = groupedby_df.drop(['donor_id','celltype','n_cells','timepoint'], axis=1)
phenotype_df = phenotype_df.transpose()
phenotype_df.head()

index,HPSI0114i-bezi_3_D30,HPSI0114i-kolf_2_D30,HPSI0114i-wegi_1_D30,HPSI0115i-aion_2_D30,HPSI0115i-aoxv_1_D30,HPSI0115i-bimq_4_D30,HPSI0115i-gost_2_D30,HPSI0115i-hecn_6_D30,HPSI0115i-iiyk_2_D30,HPSI0115i-iuad_2_D30,...,HPSI1014i-tixi_4_D52,HPSI1014i-vils_1_D52,HPSI1113i-bima_1_D52,HPSI1113i-hayt_3_D52,HPSI1113i-podx_1_D52,HPSI1113i-qolg_1_D52,HPSI1113i-qorq_1_D52,HPSI1113i-vuud_2_D52,HPSI1114i-kuul_1_D52,HPSI1213i-hehd_1_D52
AP006222.2,0.198326,0.142147,0.160197,0.213156,0.193307,0.177405,0.181125,0.219322,0.126921,0.184097,...,0.376773,0.272384,0.392572,0.301676,0.405788,0.428177,0.276859,0.375452,0.352503,0.321772
RP11-206L10.9,0.047601,0.006062,0.006713,0.012346,0.005231,0.014082,0.007296,0.015985,0.027404,0.008431,...,0.014151,0.007074,0.035446,0.009852,0.008436,0.004161,0.026287,0.013560,0.009886,0.010783
LINC00115,0.000000,0.016913,0.008016,0.006512,0.003672,0.004262,0.006904,0.017688,0.000000,0.009401,...,0.000000,0.017637,0.036590,0.007126,0.010891,0.005791,0.013481,0.011352,0.009035,0.007900
FAM41C,0.018117,0.017095,0.012269,0.023763,0.018983,0.015955,0.016163,0.011230,0.008011,0.013612,...,0.011284,0.007909,0.039070,0.010255,0.010064,0.020475,0.008824,0.012704,0.006936,0.005273
RP11-54O7.1,0.022951,0.012296,0.007431,0.026310,0.027040,0.021357,0.008420,0.007661,0.014623,0.015360,...,0.007067,0.005675,0.009553,0.018702,0.015697,0.003765,0.009385,0.044862,0.023690,0.019843


In [206]:
#number of top expressed genes - don't need to do this for interaction test - we have a feature variant filter
#selected_genes = list(phenotype_df.mean(axis=1).nlargest(n_genes).index)
#phenotype_df = phenotype_df.loc[selected_genes, :]

In [207]:
phenotype_list = phenotype_df.index
mapping_df = mapping_df.query('hgnc_symbol in @phenotype_list')
# mapping_df
mapping_df = mapping_df.drop_duplicates(subset=['hgnc_symbol'])
mapping_df = mapping_df.set_index('hgnc_symbol')

In [208]:
###### Shouldn't have to do this ######
#######################################
# limit only to hgnc symbols that map to ensembl gene IDs
phenotype_df = phenotype_df.loc[mapping_df.index,:]
phenotype_df.index = mapping_df['ensembl_gene_id']
#######################################

In [209]:
phenotype_df.head()
phenotype_df.to_csv(phenotype_file, sep='\t')

In [210]:
pc_mat = sklearn.decomposition.PCA(n_components=n_pcs).fit_transform(phenotype_df.values.transpose())
pc_df = pd.DataFrame(data=pc_mat, index=phenotype_df.columns, columns=['PC{}'.format(x) for x in range(1,n_pcs+1)])
pc_df.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
index,,,,,,,,,,
HPSI0114i-bezi_3_D30,4.864773,-3.478904,-1.127025,-1.462229,2.928072,4.251265,-2.051798,2.691721,-2.468500,-1.062393
HPSI0114i-kolf_2_D30,10.817064,7.412731,-2.619000,-3.026145,-0.512335,1.325759,-1.410847,-0.827953,0.300626,-0.924842
HPSI0114i-wegi_1_D30,6.021413,-5.418794,-1.442197,1.472833,-2.593622,0.164505,-1.204043,0.703129,1.413283,0.043769
HPSI0115i-aion_2_D30,3.449686,-2.816631,-2.814657,0.470608,0.667412,0.636077,0.689122,-1.091642,-1.547473,-0.394720
HPSI0115i-aoxv_1_D30,5.058755,-6.023935,-1.747353,1.451030,-2.188583,0.905801,-1.024874,-1.317608,0.737907,0.822012


In [223]:
### add interaction term
cov_df = pc_df.copy()
cov_df['timepoint'] = groupedby_df['timepoint'].apply(lambda x: 0 if x=='D52' else 1)
#drop PCs linearly dependent with treatment
corrs_with_treatment = cov_df.corr()['timepoint'].drop('timepoint')
correlated_covs = corrs_with_treatment.loc[lambda x: abs(x)>0.8].index
cov_df = cov_df.drop(columns=correlated_covs)
cov_df.tail()

,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,timepoint
index,,,,,,,,,,
HPSI1113i-qolg_1_D52,0.514267,1.044412,-0.974105,-0.807196,-2.316895,1.410974,0.291651,1.757880,-2.548135,0
HPSI1113i-qorq_1_D52,-2.864906,0.740195,0.182683,-3.915587,-2.100906,1.650811,0.834732,4.974053,-2.211259,0
HPSI1113i-vuud_2_D52,3.297627,0.078766,-0.466474,-1.279566,1.778665,1.705822,0.029157,-0.963616,0.700662,0
HPSI1114i-kuul_1_D52,-0.656869,1.358423,-1.586579,-0.331849,-0.203659,-0.288492,-2.417566,-1.612174,-0.551855,0
HPSI1213i-hehd_1_D52,0.228396,1.771761,-0.737801,-0.928797,-2.150059,-0.878328,1.464696,-1.359090,1.162155,0


In [224]:
cov_df.to_csv(covariates_file, sep='\t')

In [225]:
number_of_permutations = '0'
minor_allele_frequency = '0.05'
hwe = '0.001'
call_rate = '1'
window_size = '500000'
block_size = '500'
interaction_term = 'timepoint'

In [226]:
# config_dict = {'af': annotation_file,
#                'pf': phenotype_file,
#                'cf': covariates_file,
#                'kf': kinship_file,
#                'smf': sample_mapping_file,
#                'plink': genotypes_file,
#                'maf': minor_allele_frequency,
#                'hwe': hwe,
#                'np': number_of_permutations,
#                'cr': call_rate,
#                'w': window_size,
#                'bs': block_size,
#                'chunk_file': chunk_file,
#                'it': interaction_term
# }
# # write config to a file
# config_utils.write_config(config_dict, config_file)

In [227]:
python_path = 'python'
script = '/hps/nobackup/stegle/users/mjbonder/tools/hipsci_pipeline/limix_QTL_pipeline/run_interaction_QTL_analysis.py'
#script = '/nfs/software/stegle/users/dseaton/copied_from_marc/hipsci_pipeline/limix_QTL_pipeline/run_interaction_QTL_analysis.py'

In [228]:
shell_command = (
    " {python_path} {script} "
    " --plink {gen} "
    " -af {af} "
    " -pf {pf} "
    " -cf {cf} "
    " -od {od} "
    " --kinship_file {kf} "
    " --sample_mapping_file {smf} "
    " -it {interaction_term} "
#       " -gr {chunkFull} "
    " -np {np} "
#    " -wp "    #### write permutations
    " -maf {maf} "
    " -hwe {hwe} "
    " -c " #cis analysis
    " -gm standardize "
    " -w {ws} " # set window for SNPs around the gene
    " -fvf {fvf} "
    ).format(fvf=fvf_file, script=script, python_path=python_path, maf=minor_allele_frequency, hwe=hwe, np=number_of_permutations, kf=kinship_file, smf=sample_mapping_file, cf=covariates_file,od=outdir, af=annotation_file, pf=phenotype_file, gen=genotypes_file, ws=window_size, interaction_term=interaction_term)

In [229]:
postprocess_command_1=(
    "{python_path} /hps/nobackup/stegle/users/mjbonder/tools/hipsci_pipeline/post-processing_QTL/minimal_postprocess.py  "
    " -id {id} "
    " -od {od}  -oo "
    "  ".format(python_path=python_path, od=outdir, id=outdir))

postprocess_command_2=(
    "{python_path} /hps/nobackup/stegle/users/mjbonder/tools/hipsci_pipeline/post-processing_QTL/minimal_postprocess.py "
    "-id {id} "
    " -od {od} "
    " -sfo -tfb ").format(python_path=python_path, od=outdir, id=outdir)

with open(os.path.join(outdir, 'run_analysis.sh'), 'w') as f:
    for x in [shell_command, postprocess_command_1, postprocess_command_2]:
        f.write(x+'\n')